In [1]:
import warnings
warnings.simplefilter('ignore')

In [6]:
from urllib import request as req

data_dir = '/Users/ssv/newprolab/project01'
filename = 'gender_age_dataset.txt'
file_path = '/'.join([data_dir, filename])
url = 'http://data.newprolab.com/data-newprolab-com/project01/' + filename

# req.urlretrieve(url, file_path)

In [23]:
import pandas as pd
import json
from urllib.parse import urlparse
from datetime import datetime

from tqdm import tqdm_notebook

In [24]:
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 2000)

In [7]:
df = pd.read_csv(file_path, sep='\t')

In [11]:
def encode_rf(x):
    return x.encode('idna').decode('idna')  

In [ ]:
df.shape

In [8]:
df['user_json'] = df['user_json'].apply(lambda x: json.loads(x))

In [9]:
df['url_counts'] = df['user_json'].apply(lambda x: len(x['visits']))

In [ ]:
#========================================================
# Оцифровка значений пола и возраста
# Разбиение датасета 
#========================================================

In [ ]:
ages = {'18-24': 1, '25-34': 2, '35-44': 3, '45-54': 4, '>=55': 5}
df["age"] = df["age"].apply(lambda x: ages.get(x, '-'))

In [ ]:
genders = {'M': 1, 'F': 0}
df["gender"] = df["gender"].apply(lambda x: genders.get(x, '-'))


In [ ]:
test_test_df = df[df['age'] == '-'][df['gender'] == '-']

In [ ]:
test_test_df.shape

In [ ]:
df = df.drop((df[df['age'] == '-'][df['gender'] == '-']).index)

In [ ]:
#=====================================================================

In [10]:
#  Формирование столбца, содержащего массив распарсенных урлов, 
#  тип ParseResult(scheme='http', netloc='zebra-zoya.ru', 
#  path='/200028-chehol-organayzer-dlja-macbook-11-grid-it.html', 
#  params='', query='utm_campaign=397720794&utm_content=397729344&utm_medium=cpc&utm_source=begun', fragment='')

df['url_list'] = df['user_json'].apply(lambda x: [urlparse(x['url']) for x in x['visits']])

In [11]:
#  Формирование столбца, содержащего массив дат, полученных из timestamp, 
#  тип datetime.datetime(2015, 3, 24, 10, 25, 44)
df['ts_list'] = df['user_json'].apply(lambda x: \
                                       [datetime.fromtimestamp(int(float(str(x['timestamp'])[:10] + \
                                                            '.' + str(x['timestamp'])[10:13]))) \
                                        for x in x['visits']])

In [12]:
# Выбор уникальных урлов из массива распарсенных урлов, ParseResult.netloc (str)
df['url_netloc'] = df['url_list'].apply(lambda x: list(set([y.scheme + '://' + y.netloc for y in x ])))

In [13]:
#========================================================

In [14]:
# Перекодировка урлов x.encode('idna').decode('idna')
# с очисткой от мусора 
# Очищенные индексы:                 
# netloc -- 7083 ==> 
# netloc -- 12521 ==> 
# netloc -- 22669 ==> 
# netloc -- 33906 ==>
#---------------------------------------------------------
i = 0
for items in tqdm_notebook(df['url_netloc']):    
    for netloc in items:
        try:
            netloc.encode('idna').decode('idna')
        except:           
            df['url_netloc'][i].remove(netloc)
            # print('netloc -- {} ==>'.format(i), netloc)
            netloc = ''
            continue     
    i += 1

A Jupyter Widget

In [15]:
url_for_scraping = set()
for items in tqdm_notebook(df['url_netloc']):    
    for netloc in items:
        url_for_scraping.add(netloc.replace('www.', ''))
print(len(url_for_scraping))        

A Jupyter Widget


113909


In [46]:
import pickle
with open('url_for_scraping.pickle', 'wb') as file:
    pickle.dump(url_for_scraping, file)

In [ ]:
# with open('url_for_scraping.pickle', 'rb') as file:
#     url_for_scraping = pickle.load(file)

In [119]:
#----------------------------------------------------------
# Отбор ключевых слов в тегах 
# <meta content="Ноутбуки, ...." name="description"/>
# <meta content="Ноутбуки, ...." name="keywords"/>
#----------------------------------------------------------
bsObj.find('meta', {'name':'keywords'}).attrs['content']
bsObj.find('meta', {'name':'description'}).attrs['content']

'Ноутбуки, Нетбуки, Запчасти и электронные компоненты для ноутбука!'

In [127]:
df_med_keywords = pd.DataFrame()
df_med_keywords[['uid', 'url_netloc']] = df[['uid', 'url_netloc']]

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

for index, row in tqdm_notebook(df_med_keywords[:3].iterrows()):
    keywords = []
    tag = ''
    for url in row['url_netloc']:
        try:
            html = BeautifulSoup(urlopen(url).read())
        except:
            continue
        tag = html.find('meta', {'name':'description'})    
        if tag != None:
            keywords.append(' ' + tag.attrs['content'])
        tag = html.find('meta', {'name':'keywords'})    
        if tag != None:
            keywords.append(' ' + tag.attrs['content'])    
    
    df_med_keywords.iloc[index]['raw_keywords'] = keywords

NameError: name 'df_med_keywords' is not defined

In [152]:
def get_keywords(x):
    keywords = []
    tag = ''
    for url in x:
        try:
            html = BeautifulSoup(urlopen(url).read())
        except:
            continue
        tag = html.find('meta', {'name':'description'})    
        if tag != None:
            keywords.append(' ' + tag.attrs['content'])
        tag = html.find('meta', {'name':'keywords'})    
        if tag != None:
            keywords.append(' ' + tag.attrs['content']) 
    return keywords  


In [ ]:
df_med_keywords['raw_keywords'] = df['url_netloc'].apply(get_keywords)

<meta content="Ноутбуки, Нетбуки, Запчасти и электронные компоненты для ноутбука!" name="description"/>
<meta content="Ноутбуки, Нетбуки, Запчасти и электронные компоненты для ноутбука!" name="keywords"/>

In [48]:
[x.encode('idna').decode('idna') for x in netloc_list]

NameError: name 'netloc_list' is not defined

In [49]:
'xn----7sbbih4a6agbl2ah.xn--p1ai'.encode('idna').decode('idna')

'порадуй-авто.рф'

In [3]:
'gotovim-doma.ru'.encode('idna').decode('idna')

'm.news.yandex.ru'

In [276]:
ddd = datetime.fromtimestamp(int(float(str('1426661722000')[:10] + '.' + str('1426661722000')[10:13])))

In [288]:
ddd.date().day

2015

In [275]:
df['ts_list'][1000]

[datetime.datetime(2014, 12, 29, 12, 19, 38),
 datetime.datetime(2015, 3, 25, 13, 3, 50),
 datetime.datetime(2015, 3, 25, 13, 3, 50),
 datetime.datetime(2015, 3, 25, 13, 2, 53),
 datetime.datetime(2015, 3, 25, 13, 2, 53),
 datetime.datetime(2015, 3, 25, 13, 2, 25),
 datetime.datetime(2015, 3, 25, 13, 2, 25),
 datetime.datetime(2015, 3, 25, 13, 2, 20),
 datetime.datetime(2015, 3, 25, 13, 2, 20),
 datetime.datetime(2015, 3, 25, 13, 2, 11),
 datetime.datetime(2015, 3, 25, 13, 2, 11),
 datetime.datetime(2015, 3, 25, 13, 2, 8),
 datetime.datetime(2015, 3, 25, 13, 2, 8),
 datetime.datetime(2015, 3, 25, 13, 1, 56),
 datetime.datetime(2015, 3, 25, 13, 1, 56),
 datetime.datetime(2015, 3, 25, 13, 1, 51),
 datetime.datetime(2015, 3, 25, 13, 1, 51),
 datetime.datetime(2015, 3, 25, 13, 1, 4),
 datetime.datetime(2015, 3, 25, 13, 1, 4),
 datetime.datetime(2015, 3, 24, 10, 26, 14),
 datetime.datetime(2015, 3, 24, 10, 25, 56),
 datetime.datetime(2015, 3, 24, 10, 25, 50),
 datetime.datetime(2015, 3, 24,

In [ ]:
%matplotlib inline
import seaborn as sns

In [ ]:
sns_plot = sns.pairplot(df[['url_counts', 'age']], size=4)

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 16, 7
sns.distplot(df['url_counts'])

In [ ]:
df['url_counts'].value_counts().head(50).plot(kind='bar')

In [ ]:
rcParams['figure.figsize'] = 16, 7
sns.jointplot(df['age'], df['url_counts'], size=10)

In [4]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cat  %cd  %clear  %colors  %config  %connect_info  %cp  %debug  %dhist  %dirs  %doctest_mode  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %lf  %lk  %ll  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %lx  %macro  %magic  %man  %matplotlib  %mkdir  %more  %mv  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %rep  %rerun  %reset  %reset_selective  %rm  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python  %%python

In [1]:
import pickle
with open('data/df_med_keywords_non_age.pickle', 'rb') as file:
    df_med_keywords_non_age = pickle.load(file)

In [25]:
df_med_keywords_non_age['raw_keywords'].loc[36138:]

36138    [Самое лучшее бесплатное секс видео! AnySex - ...
36139    [Игрушки из Америки "Disney", Книжный интернет...
36140    [Последние новости в Яндекс.Новостях: Актуальн...
36141    [Сайт частных объявлений в Домодедове — Джази....
36142    [Большой ассортимент электроники, цифровой и б...
36143    [РџРѕСЃР»РµРґРЅРёРµ РЅРѕРІРѕСЃС‚Рё РЈРєСЂР°РёР...
36144    [Гостевая книга Динамо Брянск • Площадка для о...
36145    [Все о здоровье, развитии, питании ребенка от ...
36146                                                   []
36147    [Официальный сайт Дом.ru — быстрый Интернет, Ц...
36148                                                   []
36149    [Останні новини дня за тематикою: надзвичайні ...
36150    [2017 год Петуха - календарь и гороскоп 2017, ...
36151    [Торрент трекер Катушка.net позволяет бесплатн...
36152    [Новости о стиле жизни в Украине и во всем мир...
36153    [Сайт классической и современной литературы: ц...
36154    [Cтатьи,мнения, видео о Путине. Выборы Президе.

In [11]:
41138 - 5000

36138